In [ ]:
%pip install -U langchain langchain-core langchain-openai langchain-community python-dotenv
%pip install -U langchain-classic

In [ ]:
from typing import List
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.exceptions import OutputParserException
from langchain_classic.output_parsers.datetime import DatetimeOutputParser
from langchain_classic.output_parsers.boolean import BooleanOutputParser
from langchain_classic.output_parsers import OutputFixingParser
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("API_KEY")
base_url = os.getenv("OPENAI_ENDPOINT")
model_name = "gpt-4o-mini"
temp=0.0

llm = ChatOpenAI(
    base_url=base_url,
    api_key=api_key,
    model=model_name,
    temperature=temp
)

## **LLM Output Parsing**

**String Parser**

In [20]:
llm.invoke("hello")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 8, 'total_tokens': 18, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f97eff32c5', 'id': 'chatcmpl-Crmw4OBo40niMVhxxllnQ9KUcxpJu', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b6590-833f-7390-a9c3-259ad894efe0-0', usage_metadata={'input_tokens': 8, 'output_tokens': 10, 'total_tokens': 18, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [21]:
llm.invoke("hello").content

'Hello! How can I assist you today?'

In [23]:
parser = StrOutputParser()

In [24]:
parser.invoke(
    llm.invoke("hello")
)

'Hello! How can I assist you today?'

**Datetime**

In [28]:
response = llm.invoke(
    "Output a random datetime in %Y-%m-%dT%H:%M:%S.%fZ. "
    "Don't say anything else"
)
print(response.content)

2023-10-05T14:23:45.123456Z


In [31]:
parser = DatetimeOutputParser()
parser.invoke(
    llm.invoke(
        "Output a random datetime in %Y-%m-%dT%H:%M:%S.%fZ. "
        "Don't say anything else"
    )
)

datetime.datetime(2023, 10, 5, 14, 23, 45, 123456)

**Boolean**

In [32]:
response = llm.invoke(
    "Are you an AI? YES or NO only"
)
print(response.content)

YES


In [33]:
parser = BooleanOutputParser()
parser.invoke(
    input=llm.invoke(
        "Are you an AI? YES or NO only"
    )
)

True

In [14]:
parser.invoke(
    input=llm.invoke(
        "Are you Human? YES or NO only"
    )
)

False

## **Structured LLM Output**
- LangChain **llm.with_structured_output(structured data object)** gives us llm that produces structured output based on a given schema
- we can pass the structued data as a class of **TypedDict** or Pydantic **BaseModel**
- We can check and fix errors

#### Using TypedDict

In [35]:
from typing_extensions import Annotated, TypedDict

class UserInfo(TypedDict):
    """User's info."""
    name: Annotated[str, "", "User's name. Defaults to ''"]
    country: Annotated[str, "", "Where the user lives. Defaults to ''"]

llm_with_structure = llm.with_structured_output(UserInfo)

response = llm_with_structure.invoke(
    "My name is Henrique, and I am from Brazil"
)
print(response)

response = llm_with_structure.invoke(
    "The sky is blue"
)
print(response)

response = llm_with_structure.invoke(
    "Hello, my name is the same as the capital of the U.S.  "
    "But I'm from a country where we usually associate with kangaroos"
)
print(response)


{'name': 'Henrique', 'country': 'Brazil'}
{'name': '', 'country': ''}
{'name': 'Washington', 'country': 'Australia'}


#### Using Pydantic BaseModel

In [40]:
from pydantic import BaseModel, Field

class PydanticUserInfo(BaseModel):
    """User's info."""
    name: Annotated[str, Field(description="User's name. Defaults to ''", default=None)]
    country: Annotated[str, Field(description="Where the user lives. Defaults to ''", default=None, )]

llm_with_structure = llm.with_structured_output(PydanticUserInfo)

structured_output = llm_with_structure.invoke("The sky is blue")

structured_output


PydanticUserInfo(name='', country='')

In [41]:
from pydantic import BaseModel, Field

class PydanticUserInfo(BaseModel):
    """User's info."""
    name: Annotated[str, Field(description="User's name. Defaults to ''", default=None)]
    country: Annotated[str, Field(description="Where the user lives. Defaults to ''", default=None, )]

llm_with_structure = llm.with_structured_output(PydanticUserInfo)

structured_output = llm_with_structure.invoke("The sky is blue")

print(structured_output) # here we will not get any values because the input doesnt have name & country semantics

structured_output = llm_with_structure.invoke(
    "Hello, my name is the same as the capital of the U.S.  "
    "But I'm from a country where we usually associate with kangaroos"
)

print(structured_output.name)

print(structured_output.country)


name='' country=''
Washington
Australia


## Fixing LLM Structured Output Errors with LLM Parsers (Self-healing)

In [67]:
class Performer(BaseModel):
    """Filmography info about an actor/actress"""
    name: Annotated[str, Field(description="name of an actor/actress")]
    film_names: Annotated[List[str], Field(description="list of names of films they starred in")]

llm_with_structure = llm.with_structured_output(Performer)

response = llm_with_structure.invoke(
    "Generate the filmography for Scarlett Johansson. Top 5 only"
)
response

Performer(name='Scarlett Johansson', film_names=['Lost in Translation (2003)', 'The Avengers (2012)', 'Her (2013)', 'Lucy (2014)', 'Marriage Story (2019)'])

In [63]:
# Now we want to parse json formats back into the Pydantic object

# lets say we have two formats: a good one and a bad one
good_formatted_result = response.model_dump_json()
print(good_formatted_result)

misformatted_result = "{'name': 'Scarlett Johansson', 'film_names': ['The Avengers']}"
print(misformatted_result)

# Basic Pydantic Parsers are based on Pydantic objects:
parser = PydanticOutputParser(pydantic_object=Performer)

# now let's parse the good format
print(parser.parse(good_formatted_result))

# let's try to parse the bad format
try:
    print(parser.parse(misformatted_result)) # Generates error
except OutputParserException as e:
    print(e)

name='Scarlett Johansson' film_names=['Lost in Translation (2003)', 'The Avengers (2012)', 'Her (2013)', 'Lucy (2014)', 'Marriage Story (2019)']
Invalid json output: {'name': 'Scarlett Johansson', 'film_names': ['The Avengers']}
For troubleshooting, visit: https://docs.langchain.com/oss/python/langchain/errors/OUTPUT_PARSING_FAILURE 


In [66]:
# to fix the bad format, we create LLM-based Parser from the base Pydantic parser
smart_parser = OutputFixingParser.from_llm(parser=parser, llm=llm)

smart_parser.parse(misformatted_result)

Performer(name='Scarlett Johansson', film_names=['The Avengers', 'Lost in Translation', 'Marriage Story', 'Black Widow'])